In [70]:
import numpy as np
import pandas as pd
import re
import string
from collections import Counter

In [98]:
file = '../data/downloaded/twitter-2016train-CE.tsv'
data = pd.read_csv(file, sep='\t', header=None, usecols=[2,3])



In [99]:
data.head()

,2,3
0,-1,dear @Microsoft the newOoffice for Mac is grea...
1,-2,@Microsoft how about you make a system that do...
2,-1,I may be ignorant on this issue but... should ...
3,-1,"Thanks to @microsoft, I just may be switching ..."
4,0,If I make a game as a #windows10 Universal App...


In [100]:
data.shape

(5868, 2)

In [101]:
d = Counter()
labels = data.iloc[:,0].tolist()
for label in labels:
    d[label] += 1
d

Counter({-2: 87, -1: 662, 0: 1623, 1: 3065, 2: 431})

In [102]:
neutralIndices = []
labels = data.iloc[:,0].tolist()
for i in range(len(labels)):
    if labels[i] == -2:
        data.at[i,2] = -1
    elif labels[i] == 2:
        data.at[i,2] = 1 
    elif labels[i] == 0:
        neutralIndices.append(i)

In [103]:
processed = data.drop(neutralIndices)
processed.head()

,2,3
0,-1,dear @Microsoft the newOoffice for Mac is grea...
1,-1,@Microsoft how about you make a system that do...
2,-1,I may be ignorant on this issue but... should ...
3,-1,"Thanks to @microsoft, I just may be switching ..."
5,1,"Microsoft, I may not prefer your gaming branch..."


In [104]:
processed.shape

(4245, 2)

In [105]:
tools = processed.iloc[:,0].tolist()
print(len(tools))
de = Counter()

for label in tools:
    de[label] += 1
de

4245


Counter({-1: 749, 1: 3496})

In [60]:
isNeg = processed[2] == -1
df_try = processed[isNeg]
data2 = processed.append([df_try]*3,ignore_index=True)
data2.shape

(17568, 2)

In [61]:
labs = data2.iloc[:,0].tolist()
d = Counter()
for label in labs:
    d[label] += 1
d

Counter({-1: 9356, 1: 8212})

In [68]:
labels2 = data2.iloc[:,0].tolist()
for i in range(len(labels2)):
    if labels2[i] == -1:
        data2.at[i,2] = 0
data2

,2,3
0,0,@MargaretsBelly Amy Schumer is the stereotypic...
1,0,@dani_pitter I mean I get the hype around JLaw...
2,0,Amy Schumer at the #GQmenoftheyear2015 party i...
3,0,"Amy Schumer may have brought us Trainwreck, bu..."
4,0,I just think that sports are stupid &amp;anyon...
5,0,"If you do like Amy Schumer, I hope you went &a..."
6,0,"People are like, ""Amy Schumer? I don't like he..."
7,0,"Seriously though, somebody posted Amy Schumer ..."
8,0,They may be mean but I guess they're not stupi...
9,0,This Amy Schumer segment is now old news about...


In [81]:
class Preprocessor:
    
    def __init__(self):
        self.model = None
        
    def preprocess(self, texts):
        texts = [re.sub(r'([^\s\w]|_)+', '', text, flags=re.UNICODE) for text in texts]
        return texts

In [ ]:
p = Preprocessor()
texts = data2.iloc[:,1].tolist()
processed = p.preprocess(texts)
processed

In [89]:
texts = processed
labels = data2.iloc[:,0].tolist()
numSamples = len(texts)
with open('test.csv', 'w') as f:
    for i in range(numSamples):
        f.write(texts[i])
        f.write(',')
        f.write(str(labels[i]))
        if i < numSamples - 1:
            f.write('\n')
            

        

In [90]:
da = pd.read_csv('test.csv', header=None)
da.head()

,0,1
0,MargaretsBelly Amy Schumer is the stereotypica...,0
1,danipitter I mean I get the hype around JLaw I...,0
2,Amy Schumer at the GQmenoftheyear2015 party in...,0
3,Amy Schumer may have brought us Trainwreck but...,0
4,I just think that sports are stupid ampanyone ...,0
